# import mnist data

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

c:\users\sanjay\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
import tensorflow as tf

tf.reset_default_graph()
# inputs
x = tf.placeholder(tf.float32, shape=[None, 784])
labels = tf.placeholder(tf.float32, shape=[None, 10])

init = tf.initializers.random_normal(mean=0.0,stddev=0.1)

x_image = tf.reshape(x, [-1,28,28,1])
hidden = x_image
print(hidden)
# hidden 1
hidden = hidden
hidden = tf.layers.conv2d(hidden,filters=1,kernel_size=1,strides=(1, 1), padding='same',)
hidden = tf.nn.softplus(hidden)
print(hidden)
# hidden 2 multiplication layer using log and exp
hidden = hidden+0.01
hidden = tf.log(hidden)
hidden = tf.layers.conv2d(hidden,filters=1,kernel_size=5,strides=(2, 2), padding='same',kernel_initializer=init)
hidden = tf.exp(hidden)
hidden = tf.layers.conv2d(hidden,filters=5,kernel_size=5,strides=(1, 1), padding='same',kernel_initializer=init)
hidden = tf.nn.selu(hidden)
print(hidden)
# hidden 3
hidden = tf.layers.conv2d(hidden,filters=5,kernel_size=1,strides=(2, 2), padding='same',kernel_initializer=init)
#hidden = tf.layers.
hidden = tf.nn.selu(hidden)
print(hidden)
# hidden 4
hidden = tf.layers.conv2d(hidden,filters=30,kernel_size=3,strides=(2, 2), padding='same',kernel_initializer=init)
hidden = tf.layers.max_pooling2d(hidden,pool_size=3,strides=(2, 2), padding='same',)
hidden = tf.nn.selu(hidden)
print(hidden)
# flatten and predict
hidden = tf.layers.flatten(hidden)
hidden = tf.layers.dense(hidden, 10,kernel_initializer=init)
Pred = tf.nn.selu(hidden)

# Loss and summary
loss = tf.losses.softmax_cross_entropy(labels,Pred)
lossSummary = tf.summary.scalar('Loss', loss)
Optimizer = tf.train.AdamOptimizer().minimize(loss)

correct_prediction = tf.equal(tf.argmax(Pred,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
accuracySummary = tf.summary.scalar('accuracy', accuracy)

Tensor("Reshape:0", shape=(?, 28, 28, 1), dtype=float32)
Tensor("Softplus:0", shape=(?, 28, 28, 1), dtype=float32)
Tensor("Selu:0", shape=(?, 14, 14, 5), dtype=float32)
Tensor("Selu_1:0", shape=(?, 7, 7, 5), dtype=float32)
Tensor("Selu_2:0", shape=(?, 2, 2, 30), dtype=float32)


In [5]:
import time as Time
sess=tf.Session()  
sess.run(tf.global_variables_initializer())
train_writer = tf.summary.FileWriter('summary\\'+Time.strftime('%H_%M_%S'), sess.graph)
for i in range(20000):
    feed_dict={x:mnist.test.images, labels:mnist.test.labels}
    _, lossS ,accSumm= sess.run([Optimizer, lossSummary,accuracySummary], feed_dict=feed_dict) 
    train_writer.add_summary(lossS, i)
    train_writer.add_summary(accSumm, i)

KeyboardInterrupt: 

In [7]:
saver = tf.train.Saver()
save_path = saver.save(sess, "/Model/model.ckpt")